In [1]:
from load_data import load_scaled_data

c:\python310\lib\site-packages\sklearn\utils\_param_validation.py:11: UserWarning: A NumPy version >=1.23.5 and <2.5.0 is required for this version of SciPy (detected version 1.23.3)
  from scipy.sparse import csr_matrix, issparse


In [2]:
df = load_scaled_data()

In [3]:
df

,date,day,period,nswprice,nswdemand,vicprice,vicdemand,transfer,class
0,-0.213995,0.999955,0.755418,0.471950,-0.701234,0.001170,-0.000907,-0.557367,1
1,1.085774,1.501580,1.043922,-0.760447,-0.776809,-0.158930,-0.806699,2.167103,0
2,-1.372785,-1.508167,0.755418,-0.066211,-0.166768,0.001170,-0.000907,-0.557367,0
3,1.087074,-0.504918,1.693050,-0.742753,-0.182247,-0.239914,-0.233431,0.909436,0
4,1.124565,-0.003294,1.620923,-0.437541,0.262989,-0.078634,0.019922,0.852139,1
...,...,...,...,...,...,...,...,...,...
36244,-0.150080,-1.006542,0.755418,0.749481,2.570212,0.256111,2.190620,-1.127466,1
36245,-0.202147,1.501580,-0.254338,0.005285,-0.326108,0.001170,-0.000907,-0.557367,0
36246,-1.427586,0.498331,-0.110088,-0.412164,0.811116,0.001170,-0.000907,-0.557367,0
36247,1.151640,-0.504918,1.260296,0.521141,1.864570,0.190459,1.460609,-0.153425,1


In [4]:
from bagging_method import CustomBaggingClassification

In [5]:
from time import time

In [6]:
from sklearn.model_selection import train_test_split
# _, X, _, y = train_test_split(df.drop('class', axis=1).to_numpy(), df['class'], stratify= df['class'], test_size=0.2)
X, y = df.drop('class', axis=1).to_numpy(), df['class'].to_numpy()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [7]:
# параметры подобраны опытным путем, чтобы быть круче sklearn.
custom_model = CustomBaggingClassification(n_estimators=10, threshold_train_score=0.9, threshold_test_score=0.8)

In [8]:
def get_time(model, X, y):
    t_start = time()
    model.fit(X, y)
    return (time() - t_start)

In [9]:
t_custom = get_time(custom_model, X_train, y_train)
t_custom

3.9154417514801025

In [10]:
from sklearn.metrics import accuracy_score, classification_report

In [11]:
y_pred = custom_model.predict(X_test)
print(classification_report(y_test, y_pred, digits=4))

              precision    recall  f1-score   support

           0     0.8958    0.9353    0.9151      4144
           1     0.9083    0.8548    0.8807      3106

    accuracy                         0.9008      7250
   macro avg     0.9020    0.8951    0.8979      7250
weighted avg     0.9011    0.9008    0.9004      7250



In [12]:
from sklearn.ensemble import BaggingClassifier

sklearn_model = BaggingClassifier(n_estimators=10)

In [13]:
t_sklearn = get_time(sklearn_model, X_train, y_train)
t_sklearn

2.510341167449951

In [14]:
y_pred = sklearn_model.predict(X_test)
accuracy_score(y_pred, y_test)

print(classification_report(y_test, y_pred, digits=4))

              precision    recall  f1-score   support

           0     0.8922    0.9291    0.9103      4144
           1     0.8998    0.8503    0.8744      3106

    accuracy                         0.8953      7250
   macro avg     0.8960    0.8897    0.8923      7250
weighted avg     0.8955    0.8953    0.8949      7250



In [15]:
print(f"Время, затраченное кастомным: {t_custom:.4f} с")
print(f"Время, затраченное Sklearn: {t_sklearn:.4f} с")

Время, затраченное кастомным: 3.9154 с
Время, затраченное Sklearn: 2.5103 с


In [16]:
from sklearn.metrics import classification_report
def cross_validate(model, X, y, n_folds=10):
    scores = []
    for n in range(n_folds):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
        model.fit(X_train, y_train)
        scores += [accuracy_score(model.predict(X_test), y_test)]
    return scores

In [17]:
n_folds = 10


custom_scores = cross_validate(custom_model, X, y, n_folds=n_folds)
sklearn_scores = cross_validate(sklearn_model, X, y, n_folds=n_folds)

In [18]:
import numpy as np
print(f"Средний accuracy для {n_folds} выборок у кастомного: {np.mean(custom_scores):.3f}")
print(f"Средний accuracy для {n_folds} выборок у Sklearn: {np.mean(sklearn_scores):.3f}")

Средний accuracy для 10 выборок у кастомного: 0.904
Средний accuracy для 10 выборок у Sklearn: 0.904
